In [1]:
import numpy as np
import pandas as pd
import cp_optimizer
from scipy.interpolate import interp1d
import utils
import cp_optimizer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
q_list = ["q1", "q3", "q5", "q4","q2", "q5", "q1"]

res = utils.interpolate(utils.load(q_list, num_partitions='16'))

precision = 1000
C =16
C_ = None # for split
Q = len(q_list)
R = int(round(Q / 2))
# reg_factor = 0.001

In [3]:
path_sets_idx = utils.get_path_sets(range(len(q_list)))
num_paths = len(path_sets_idx)
probas = np.random.random(size=num_paths)
probas = list(probas / sum(probas))

proba_variables = cp_optimizer.get_proba_variables(q_list, probas)

In [4]:
cp_optimizer.compute_result(q_list, res, C, R, precision, C_=None, proba_variables=None)

(1.5017979145050049,
 (30.195,
  [[('q1', 8), ('q1', 8)],
   [('q5', 8), ('q5', 8)],
   [('q3', 8), ('q4', 4), ('q2', 4)]]))

## Split section

In [5]:
MAX_LEN_QUERIES = 14
def split(q_list, res, C, precision, proba_variables=None, normal_exec=True,reg_factor=None):
    if len(q_list) > MAX_LEN_QUERIES:
        R = 2
        Q = len(q_list)
        C_ = 2 * Q
        process_time, (runtime, res_schedule) = cp_optimizer.compute_result(q_list, res, C, R, precision, name_queries=False, 
                                                                            C_=C_, split=True, proba_variables=proba_variables, 
                                                                            normal_exec=normal_exec, reg_factor=reg_factor)

        # print("success until there")
        # print(res_schedule)
        q_left, process_time_left = split([x[0] for x in res_schedule[0]], res, C, precision)
        q_right, process_time_right = split([x[0] for x in res_schedule[1]], res, C, precision)

        total_time = process_time + process_time_left + process_time_right
        if proba_variables is None:
            return q_left + q_right, total_time
        else:
            #TODO : correct when recursive [0]
            return rearrange_queries_probas(q_list, q_left[0], q_right[0], proba_variables), total_time
    else:
        return [q_list], 0

In [12]:
q_list = ["q1", "q3", "q5", "q4","q2", "q5", "q1",
         "q1", "q3", "q5", "q4","q2", "q5", "q1", "q1"]

res = utils.interpolate(utils.load(q_list, num_partitions='16'))

precision = 1000
C =16
C_ = None # for split
Q = len(q_list)
R = int(round(Q / 2))
# reg_factor = 0.001

path_sets_idx = utils.get_path_sets(range(len(q_list)))
num_paths = len(path_sets_idx)
np.random.seed(0)
probas = np.random.random(size=num_paths)
probas = list(probas / sum(probas))

proba_variables = cp_optimizer.get_proba_variables(q_list, probas)

In [13]:
# normal exec
l_n = [0, 1, 2, 3, 4, 6, 7, 9, 10, 13, 14]
r_n = [5, 8, 11, 12]


# alternative exec
l = [0, 1, 2, 4, 5, 6, 9, 12, 13]
r = [3, 7, 8, 10, 11, 14]


In [14]:
utils.print_tree(q_list, probas, index=True)

                                           0

                            1                            2                            

                 3                 4                 5                 6                 

         7         8         9         10         11         12         13         14         

     0.114085   0.148671   0.1253   0.113268   0.088068   0.134266   0.090964   0.185378   


In [49]:
def rearrange_queries_probas(q_list, q_left, q_right, proba_variables):
    _, _, path_sets_idx = proba_variables
    left_queries = set(q_left)
    right_queries = set(q_right)
    # 1. split paths : determine which belong only to left
    left_paths = []
    right_paths_raw = []
    for p in path_sets_idx:
        intersect = p.intersection(right_queries)
        if len(intersect) == 0:
            left_paths.append(p)
        else:
            right_paths_raw.append(p)

    print("paths left", left_paths)
    print("paths right", right_paths_raw)
    # 2. select queries appearing only in right paths
    remain_queries = left_queries.difference(set.union(*left_paths))
    print("remain", remain_queries)
    left_queries = left_queries.difference(remain_queries)
    print("left", left_queries)
    right_queries = right_queries.union(remain_queries)
    print("right", right_queries)

    # 3. redefine paths right containing queries in left
    right_paths = []
    for p in right_paths_raw:
        new_p = right_queries.intersection(p)
        right_paths.append(new_p)

    # 4. redefine left and right q_list
    # q_list_left = [q_list[i] for i in sorted(left_queries)]
    # q_list_right = [q_list[i] for i in sorted(right_queries)]
    q_list_left = list(left_queries)
    q_list_right = list(right_queries)
    
    return (q_list_left, left_paths), (q_list_right, right_paths)

In [50]:
# alternative exec
l_a = [0, 1, 2, 3, 5, 6, 8, 12, 14]
r_a = [4, 7, 9, 10, 11, 13]

#normal exec
# l_n = [0, 1, 2, 3, 4, 6, 7, 9, 10, 13, 14]
# r_n = [5, 8, 11, 12]

In [51]:
rearrange_queries_probas(q_list, l_a, r_a, proba_variables)

paths left [{0, 1, 3, 8}, {0, 2, 12, 5}, {0, 2, 6, 14}]
paths right [{0, 1, 3, 7}, {0, 1, 4, 9}, {0, 1, 10, 4}, {0, 2, 11, 5}, {0, 2, 13, 6}]
remain set()
left {0, 1, 2, 3, 5, 6, 8, 12, 14}
right {4, 7, 9, 10, 11, 13}


(([0, 1, 2, 3, 5, 6, 8, 12, 14], [{0, 1, 3, 8}, {0, 2, 5, 12}, {0, 2, 6, 14}]),
 ([4, 7, 9, 10, 11, 13], [{7}, {4, 9}, {4, 10}, {11}, {13}]))

In [18]:
# split(q_list, res, C, precision, proba_variables=None, reg_factor=None)
# split(q_list, res, C, precision, proba_variables=proba_variables, reg_factor=None, normal_exec=True)